# Решение задачи регрессии с помощью нейронных сетей.
# Предсказание стоимости ноутбуков по их характеристикам.


#  Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder # Метод кодирования категориальных переменных

from tensorflow.keras.models import Sequential # Модель последовательной НС
from tensorflow.keras.layers import Dropout, Normalization # Dropout против переобучения, нормализация данных
from tensorflow.keras.optimizers import Adam # Оптимизатор
from tensorflow.keras import layers, regularizers # Настройка архитектуры НС, регуляризация против переобчуния

#  Предобработка данных

## Смотрим содержание датасета

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/laptop_price.csv', encoding='latin-1')
data.head(10)

NameError: name 'pd' is not defined

Можно видеть, что в датасете довольно много столбцов с категориальными данными, требующими предварительной обработки и перевода в численные.

In [ ]:
data.info()

NameError: name 'data' is not defined

Датасет состоит из 13 столбцов (один из них хранит в себе тип int, два - тип float, остальные десять - тип object) и 1303 строк. В столбцах отсутствуют пропущенные значения (тип NULL).

Посмотрим на уникальные значения в различных столбцах с категориальными данными, требующих обработки:

In [ ]:
data['Company'].value_counts()

In [ ]:
data['Product'].value_counts()

Столбец Product содержит слишком много уникальных значений (618) относительно количесвта строк в датасете (1303), поэтому в дальнейшем удалим его.

In [ ]:
data['TypeName'].value_counts()

In [ ]:
data['ScreenResolution'].value_counts()

In [ ]:
data['Cpu'].value_counts()

In [ ]:
data['Memory'].value_counts()

In [ ]:
data['Gpu'].value_counts()

In [ ]:
data['OpSys'].value_counts()

Посмотрим на распределение цены в датасете:

In [ ]:
data['Price_euros'].describe()

In [ ]:
plt.hist(data['Price_euros'], bins=30, color='blue', alpha=0.7)
plt.xlabel('Цена ноутбуков')
plt.ylabel('Количество ноутбуков')
plt.title('Распределение цен')
plt.show()

NameError: name 'plt' is not defined

## Непосредственная обработка данных

Для обработки данных были приняты следующие решения:

*   Разбить столбец ScreenResolution на столбцы IPS (0/1), Touchscreen (0/1), res_X и res_Y (разрешение экрана по горизонтали и вертикали)
* Разбить столбец Cpu на столбцы Cpu_name и Cpu_freq (частота)
*В столбце Memory перевести всё в GB и разбить на четыре столбца SSD, HDD, Hybrid, Flash Storage
*Убрать GB и kg в столбцах Ram и Weight

In [ ]:
# Обработка столбца ScreenResolution
data['Ips']=data['ScreenResolution'].apply(lambda x:1 if 'IPS' in x else 0) # Есть IPS? 0/1
data['Touchscreen']=data['ScreenResolution'].apply(lambda x:1 if 'Touchscreen' in x else 0) ## Есть Touchscreen? 0/1
var=data['ScreenResolution'].str.split('x',n=1,expand=True) # Разбиение разрешения экрана на горизонтальную и вертикальную составляющие
data['res_X']=var[0]
data['res_Y']=var[1]
data['res_X']=data['res_Y'].str.replace(',','').str.findall(r'(\d+\.?\d+)').apply(lambda x:x[0])
data[['res_Y','res_X']]=data[['res_Y','res_X']].astype(int)
data.drop('ScreenResolution', axis=1, inplace=True) # Удаляем первоначальный столбец

# Обработка столбца Cpu
data['Cpu_name']=data['Cpu'].apply(lambda x:" ".join(x.split()[0:3])) # Название процессора
data['Cpu_freq']=data['Cpu'].apply(lambda x:" ".join(x.split()[-1:])) # Частота процессора
data['Cpu_freq']=data['Cpu_freq'].str.replace('GHz', '').astype(float) # Убираем размерность GHz
data.drop('Cpu', axis=1, inplace=True) # Удаляем первоначальный столбец

# Перевод столбца Memory в GB
data['Memory'] = data['Memory'].str.replace('TB', '000GB')

# Замена записи по типу 256GB SSD + 256GB SSD на 512GB SSD (суммирование памяти одинакого типа)
data['Memory'] = data['Memory'].str.replace('256GB SSD +  256GB SSD', '512GB SSD')
data['Memory'] = data['Memory'].str.replace('512GB SSD +  256GB SSD', '768GB SSD')
data['Memory'] = data['Memory'].str.replace('512GB SSD +  512GB SSD', '1024GB SSD')
data['Memory'] = data['Memory'].str.replace('1000GB HDD +  1000GB HDD', '2000GB HDD')

# Создание столбцов для каждого типа памяти
data['SSD'] = data['Memory'].str.extract('(\d+)GB SSD').fillna(0).astype(int) # Столбец SSD
data['HDD'] = data['Memory'].str.extract('(\d+)GB HDD').fillna(0).astype(int) # Столбец HDD
data['Flash Storage'] = data['Memory'].str.extract('(\d+)GB Flash Storage').fillna(0).astype(int) # Столбец Flash Storage
data['Hybrid'] = data['Memory'].str.extract('(\d+)GB Hybrid').fillna(0).astype(int) # Столбец Hybrid
data.drop('Memory', axis=1, inplace=True) # Удаляем первоначальный столбец

data['Ram']=data['Ram'].apply(lambda x:int(x[0])) # Убираем размерность GB в столбце Ram
data['Weight'] = data['Weight'].astype(str).str.replace('kg', '').astype(float) # Убираем размерность kg в столбце Weight

# Удаляем первоначальные столбцы
data.drop('Gpu', axis=1, inplace=True)
data.drop('laptop_ID', axis=1, inplace=True)
data.drop('Product', axis=1, inplace=True)

In [ ]:
data

Для преобразования категориалных данных в численный вид был выбран тип кодирования one-hot encoding из библиотеки sklearn (создаёт столбцы для всех уникальных категорий и расставляются 1 и 0 соответственно).

In [ ]:
#Получаем все категориальные столбцы
cat_columns = data.select_dtypes(['object']).columns

encoder = OneHotEncoder(handle_unknown='ignore')

#Переводим категориальные столбцы в числовые (бинарные)
encoder_data = pd.DataFrame(encoder.fit_transform(data[cat_columns]).toarray())

#Присоединяем преобразованные столбцы к датасету
data = data.join(encoder_data)

# Удаляем изначальные категориальные столбцы
data.drop(cat_columns, axis= 1 , inplace= True )

In [ ]:
data

In [ ]:
data.describe()

# Нейронная сеть

## Создание тренировочной и тестовой выборки

In [ ]:
train_dataset = data.sample(frac=0.70, random_state=10) # 85% - тренировочная выборка
test_dataset = data.drop(train_dataset.index) # остальные 15% - тестовая выборка

X_train = train_dataset.copy()
X_test = test_dataset.copy()

y_train = X_train.pop('Price_euros')
y_test = X_test.pop('Price_euros')

## Архитектура нейросети

In [ ]:
normalizer = Normalization(axis=-1) # Слой нормализации входных данный
normalizer.adapt(np.array(X_train))

model = Sequential([
      normalizer,
      layers.Dense(300, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
      layers.Dropout(0.5),
      layers.Dense(1200, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
      layers.Dropout(0.5),
      layers.Dense(200, activation='relu', kernel_regularizer=regularizers.l2(0.0001)),
      layers.Dense(1, activation='linear')
      ])

model.compile(loss='mse',
                optimizer = Adam(0.001),
                metrics=['mae'])

## Обучение нейросети

In [ ]:
model.summary()

history = model.fit(
    X_train,
    y_train,
    validation_split=0.1,
    verbose=1, epochs=200)

plt.plot(history.history['mae'],
         label='Средняя абсолютная ошибка на обучающем наборе')
plt.plot(history.history['val_mae'],
         label='Средняя абсолютная ошибка на валидационном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Средняя абсолютная ошибка')
plt.legend()
plt.show()

In [ ]:
model.evaluate(X_test, y_test, verbose=0)

## Результат на тестовой выборке

In [ ]:
y_predict = model.predict(X_test).flatten()
abs(y_test - y_predict).describe()

In [ ]:
plt.figure(figsize=(6,6))
plt.hist(abs(y_test - y_predict), bins=50, color='blue', alpha=0.7)
plt.xlabel('Размер ошибки')
plt.ylabel('Частота')
plt.title('Распределение ошибки')
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(range(0, len(y_test)), y_test, color="green", label="Тестовые значения")
plt.scatter(range(0, len(y_test)), y_predict, label="Предсказанные значения")
plt.xlabel("Номер ноутбука")
plt.ylabel("Стоимость (евро)")
plt.legend()
plt.show()